In [1]:
# 앞 절에서 수행한 것들

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

dataset_root = os.path.join(os.getcwd(), "datasets")
housing_path = os.path.join(dataset_root, "housing")

def load_housing_data(housing_path=housing_path):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)


housing = load_housing_data()
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

split = StratifiedShuffleSplit(n_splits=1,
                               test_size=0.2,
                               random_state=42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    

for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Prepare the Data for Machine Learning Algorithms

머신러닝 알고리즘을 위한 데이터를 준비해본다. 이는 수동으로 작업하기보다, 함수로 작성하는 것이 더욱 좋다. 그 이유는 다음과 같다.
- 다른 dataset에도 동일한 변환을 재사용 할 수 있다.
- 향후 프로젝트에 재사용할 변환 함수들의 라이브러리를 점진적으로 구축하게 된다.
- live system의 알고리즘에 새로운 데이터를 주입하기 전에 변환을 하기 위해 이 함수를 사용할 수있다.
- 다양한 변환들을 쉽게 시도하고, 가장 잘 동작하는 변환들의 조합을 확인할 수 있게 만들어 줄 것이다.

`strat_train_set`으로 다시 되돌아가서, training set에서 변환이 일어나면 안되는 target을 분리한다.

In [2]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

### Data Cleaning

여기서는 missing feature(`total_bedrooms`의 non-null value)를 처리하는 함수를 만들어본다. 이는 다음과 같은 3가지 방법으로 수행할 수 있다.
1. missing feature인 district를 모두 제거한다.
2. 해당 feature 전체를 제거한다.
3. 직접 값을 설정해준다.(0, mean, median 등)

위의 3가지 방법은 DataFrame의 `dropna()`, `drop()`, `fillna()`를 통해 다음과 같이 쉽게 수행할 수 있다.

    # 1번째 방법
    housing.dropna(subset=["total_bedrooms"])

    # 2번째 방법
    housing.drop("total_bedrooms", axis=1)

    # 3번째 방법(여기서는 median으로 한 예)
    median = housing["total_bedrooms"].median()
    housing["total_bedrooms"].fillna(median, inplace=True)

만약, 3번째 방법으로 수행한다면, 반드시 training set에서 계산한 값으로 설정해야한다. 또한, 이후 evaluation에 사용되는 test set과 시스템의 운영시에 새로운 데이터의 missing value에도 이 값을 사용해야 하므로 반드시 저장해놓아야 한다.

사이킷런의 `SimpleImputer` class를 사용하면 missing value를 쉽게 다룰 수 있다. 먼저, 다음과 같이 `strategy`를 constructor의 파라미터로 전달하며 `SimpleImputer` instance를 생성한다.

In [3]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

median 값은 수치 값에서만 계산될 수 있으므로, text feature인 `ocean_proximity`를 제거해준다.

In [4]:
housing_num = housing.drop("ocean_proximity", axis=1)

이제 위 수치 값으로 이루어진 데이터를 인자로 넘겨주며 `fit()`을 해주게되면, imputer instance는 각 feature별로 median을 계산해서 `statistics_` instance 변수에 저장한다.

현재의 데이터에는 `total_bedrooms`에만 missing value가 있지만, 새로운 데이터에는 어떤 값이 missing될지 모르므로, 모든 feature에 대해 계산해놓는 것이 안전하다.

In [5]:
imputer.fit(housing_num)
print(imputer.statistics_)

# median값이 계산이 잘 되었는지 확인
print(housing_num.median().values)

[-118.51     34.26     29.     2119.5     433.     1164.      408.
    3.5409]
[-118.51     34.26     29.     2119.5     433.     1164.      408.
    3.5409]


이제 위의 학습된 `imputer`를 통해 training set의 missing value를 median값으로 대체할 수 있다.

In [6]:
X = imputer.transform(housing_num)

`transform()`의 결과는 numpy array형태이다. 다시 pandas의 DataFrame으로 바꾸고 싶다면 다음과 같이 수행한다.

In [7]:
print(type(X))
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
print(type(housing_tr))
housing_tr.describe()

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
count,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000
mean,-119.575834,35.639577,28.653101,2622.728319,533.998123,1419.790819,497.060380,3.875589
std,2.001860,2.138058,12.574726,2138.458419,410.839621,1115.686241,375.720845,1.904950
min,-124.350000,32.540000,1.000000,6.000000,2.000000,3.000000,2.000000,0.499900
25%,-121.800000,33.940000,18.000000,1443.000000,296.000000,784.000000,279.000000,2.566775
50%,-118.510000,34.260000,29.000000,2119.500000,433.000000,1164.000000,408.000000,3.540900
75%,-118.010000,37.720000,37.000000,3141.000000,641.000000,1719.250000,602.000000,4.744475
max,-114.310000,41.950000,52.000000,39320.000000,6210.000000,35682.000000,5358.000000,15.000100


## Handling Text and Categorical Attributes

이제 text로 구성된 categorical feature인 `ocean_proximity`를 처리해보자.

In [8]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

,ocean_proximity
17606,<1H OCEAN
18632,<1H OCEAN
14650,NEAR OCEAN
3230,INLAND
3555,<1H OCEAN
19480,INLAND
8879,<1H OCEAN
13685,INLAND
4937,<1H OCEAN
4861,<1H OCEAN


대부분의 머신러닝 알고리즘은 숫자를 다루므로, text를 숫자로 바꾸어준다. 다음과 같이 사이킷런의 `OrdinaryEncoder` class를 사용하면 된다.

In [9]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
print(housing_cat_encoded[:10])

[[0.]
 [0.]
 [4.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]]


category들의 목록은 `categories_` instance 변수를 통해 출력할 수 있다.

In [10]:
print(ordinal_encoder.categories_)

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype=object)]


위 방법의 문제는 머신러닝 알고리즘이 가까운 값을 먼 값보다 비슷하다고 생각한다는 것이다. 예를 들어, 0과 1은 0과 4보다 유사하다고 판단한다.

이를 해결하기 위한 일반적인 솔루션은 각 category별 binary 값으로 표현하는 것이다. 이 방법은 **one-hot encoding**이라고 한다.

사이킷런에서는 `OneHotEncoder` class를 통해 categorical value를 one-hot vector로 표현할 수 있다. 

In [11]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

여기서 주목할 점은 출력이 numpy array가 아닌 scipy sparse matrix라는 것이다. 이는 매우 많은 category를 가지는 categorical feature에서 매우 유용하다. 0과 1로만 이루어진 아주 큰 matrix는 메모리 낭비가 심하므로, 1의 위치만을 가지는 sparse matrix 형태로 저장하는 것이다.

만약 이 sparse matrix형태를 원래의 dense matrix(numpy array)로 변환하려면, 다음과 같이 `toarray()`를 사용하면 된다.

In [12]:
housing_cat_1hot.toarray()

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

## Custom Transformers

사이킷런에서 직접 custom transformer를 만들기 위해서는 3가지 methods:`fit()`, `transform()`, `fit_transform()`가 구현된 class만 작성하면 된다.

또한 `fit_transform()`은 `TransformerMixin`  class만 상속해주면 자동으로 구현되며, `BaseEstimator` class를 추가로 상속할 경우, hyperparameter tuning의 자동화에 유용한 2가지 methods:`get_params()`와 `set_params()`가 구현된다.

여기서는 combined feature를 추가하는 transformer를 만들어본다.

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_idx, bedrooms_idx, population_idx, households_idx = 3, 4, 5, 6

class CombinedFeaturesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        # Nothing else to do
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_idx]/X[:, households_idx]
        population_per_household = X[:, population_idx]/X[:, households_idx]
        # bedrooms_per_room은 선택
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_idx]/X[:, rooms_idx]
            return np.c_[X,
                         rooms_per_household,
                         population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X,
                         rooms_per_household,
                         population_per_household]
    

feature_adder = CombinedFeaturesAdder(add_bedrooms_per_room=False)
housing_extra_features = feature_adder.transform(housing.values)

위 예제에서는 `add_bedrooms_per_room` hyperparameter를 하나 가지고 있으며 기본으로 `True`를 설정했다.(합리적인 기본 값을 설정하는 것이 좋음)

이처럼 100% 확신이 없는 데이터에 대해 hyperparameter를 추가하면, 더욱 많은 combination을 자동으로 시도할 수 있게 되며, 최상의 combination을 찾게 될 가능성을 높여준다.

## Feature Scaling

몇가지 경우를 제외하고, 머신러닝 알고리즘은 서로 다른 scale의 수치 feature에서 좋은 성능을 내지 못하므로 feature scaling은 아주 중요하다.

모든 feature가 같은 scale을 가지도록 하는 일반적인 방법은 다음의 2가지가 있다.

#### 1. min-max scaling

min-max scaling은 다음의 식을 통해 수행된다.
- $x = \dfrac{x-x_{min}}{x_{max}-x_{min}}$

min-max scaling은 값을 0에서 1사이의 범위로 만들어준다.

사이킷런에서는 `MinMaxScaler`를 통해 수행할 수 있으며, `feature_range` hyperparameter를 통해 0에서 1이 아닌 다른 범위를 지정할 수도 있다.

#### 2. standardization

Standardization은 다음의 식을 통해 수행된다.
- $x = \dfrac{x-x_{mean}}{x_{std}}$

standardization을 수행하면, 평균이 0이고 분산이 1인 분포를 가지게 되며 값의 범위가 정해지지는 않는다. 또한, standardization은 min-max scaling에 비해 outlier에 덜 민감하다는 특징이 있다.

사이킷런에서는 `StandardScaler`를 통해 수행할 수 있다.

### 참고
지금까지 살펴본 모든 transformer는 training set에서 값을 구한다는 것을 기억해야한다.

즉, training set에 `fit()`을 한 후, 모든 데이터(train, test, new data 등)에 `transform()`한다.

## Transformation Pipelines

사이킷런의 `Pipeline` class를 사용하면 다음과 같이 여러 변환 sequence를 올바른 순서로 실행할 수 있다.

이제 앞서 해왔던 변환들을 하나의 pipeline으로 만들어보자.

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('features_adder', CombinedFeaturesAdder()),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

`Pipeline` constructor는 (이름, estimator)쌍으로 이루어진 list를 받아 sequence를 정의한다. 또한, 여기서 정의된 이름은 나중에 hyperparameter tuning에서 사용된다.

pipeline의 `fit()`을 호출하면, estimator들은 순서대로 `fit_transform()`을 호출하며 마지막 estimator는 `fit()`만 호출한다. 

따라서, 마지막 estimator는 `fit()`, 나머지 estimator들은 `fit()`, `transform()` 또는 `fit_transform()`까지 포함해서 가지고 있어야 한다. (`fit()`, `transform()`만 있는 경우, `fit_transform()`을 호출하면, 둘을 차례로 실행한다.)

pipeline은 마지막 estimator의 method를 제공한다. 여기서는 `StandardScaler`에 `fit()` ,`transform()`이 존재하므로, `fit_transform()`을 사용할 수 있었다.

사이킷런의 `ColumnTransformer`는 각 column별로 처리해야 할 여러 변환들을 하나로 묶어준다. 이를 사용하면 다음과 같이 categorical feature와 numerical feature를 처리하는 변환을 하나로 묶을 수 있다.

In [15]:
from sklearn.compose import ColumnTransformer

num_features = list(housing_num)
cat_features = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", OneHotEncoder(), cat_features),
])

housing_prepared = full_pipeline.fit_transform(housing)